In [1]:
# PyTorch Modules
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision import models
from torchvision import transforms
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Other non-PyTorch Modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from matplotlib.pyplot import imshow
import matplotlib.pylab as plt
from PIL import Image
import time
from datetime import datetime
import pickle
import json
import random

In [2]:
torch.cuda.empty_cache()

In [3]:
dataDF = pd.read_csv('data_fold.csv')
dataDF = dataDF.set_index('SOPInstanceUID')

In [4]:
dataDF.columns

Index(['StudyInstanceUID', 'SeriesInstanceUID', 'pe_present_on_image',
       'negative_exam_for_pe', 'qa_motion', 'qa_contrast', 'flow_artifact',
       'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1', 'leftsided_pe', 'chronic_pe',
       'true_filling_defect_not_pe', 'rightsided_pe', 'acute_and_chronic_pe',
       'central_pe', 'indeterminate', 'window_center', 'window_width',
       'intercept', 'slope', 'slice_thickness', 'kvp', 'ma', 'exposure',
       'img_pos', 'conv_kernel', 'patient_position', 'pixel_spacing',
       'bits_stored', 'high_bit', 'img_count', 'fold'],
      dtype='object')

In [5]:
dataDF = dataDF.reindex(columns=['StudyInstanceUID', 'SeriesInstanceUID', 'pe_present_on_image', 'negative_exam_for_pe',
                       'indeterminate', 'chronic_pe', 'acute_and_chronic_pe', 'central_pe', 'leftsided_pe',
                       'rightsided_pe', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1','fold','img_pos','patient_position',
                       'intercept', 'slope'])                       

In [6]:
dataDF.head()

,StudyInstanceUID,SeriesInstanceUID,pe_present_on_image,negative_exam_for_pe,indeterminate,chronic_pe,acute_and_chronic_pe,central_pe,leftsided_pe,rightsided_pe,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,fold,img_pos,patient_position,intercept,slope
SOPInstanceUID,,,,,,,,,,,,,,,,,
c0f3cb036d06,6897fa9de148,2bfbb7fd2e8b,0,0,0,0,0,0,1,1,0,1,3,-234.5,HFS,-1024,1
f57ffd3883b6,6897fa9de148,2bfbb7fd2e8b,0,0,0,0,0,0,1,1,0,1,3,-252.5,HFS,-1024,1
41220fda34a3,6897fa9de148,2bfbb7fd2e8b,0,0,0,0,0,0,1,1,0,1,3,-432.5,HFS,-1024,1
13b685b4b14f,6897fa9de148,2bfbb7fd2e8b,0,0,0,0,0,0,1,1,0,1,3,-434.5,HFS,-1024,1
be0b7524ffb4,6897fa9de148,2bfbb7fd2e8b,0,0,0,0,0,0,1,1,0,1,3,-436.5,HFS,-1024,1


In [7]:
dataDF.to_csv('for_reference.csv')

In [8]:
trainDF = dataDF[dataDF['fold']!=0]
valDF = dataDF[dataDF['fold']==0]

In [ ]:
embeddingDirPath = 'data/embeddings/CNNmodel_01_cv0_epoch1_20201014_0012.pth/val/'

In [ ]:
class embeddingsDataset(Dataset):
    """create sample dataset to work with"""

    def __init__(self, dataDF = None, listOfStudies = None):
        self.dataDF = dataDF
        self.listOfStudies = listOfStudies

    def __len__(self):
        return len(self.listOfStudies)

    def __getitem__(self, idx):
        embedDict = pickle.load(open(embeddingDirPath+self.listOfStudies[idx]+'.p', 'rb'))
        embeddingVolume = np.array(embedDict['embeddings'])
        listOfImages = embedDict['ids']
        imageLevelLabels = [self.dataDF.loc[eachImageID, 'pe_present_on_image']for eachImageID in listOfImages]
        imageLevelLabels = np.array(imageLevelLabels).astype(np.float32)
        studyLevelLabels = self.dataDF.loc[listOfImages[0]][3:12].values
        studyLevelLabels = np.array(studyLevelLabels).astype(np.float32)
        return embeddingVolume, (imageLevelLabels, studyLevelLabels)

In [ ]:
trainEmbeddingsDataset = embeddingsDataset(dataDF=dataDF, listOfStudies=trainDF['StudyInstanceUID'].unique())
trainEmbeddingsDataloader = DataLoader(trainEmbeddingsDataset, batch_size=1, shuffle=True, num_workers=1)

valEmbeddingsDataset = embeddingsDataset(dataDF=dataDF, listOfStudies=valDF['StudyInstanceUID'].unique())
valEmbeddingsDataloader = DataLoader(valEmbeddingsDataset, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
INPUT_SIZE = 64
HIDDEN_SIZE = 32
NUM_LAYERS = 1
NUM_CLASSES = 1

class BiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout = 0.1):
        super(BiGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.GRU = nn.GRU(
            input_size, hidden_size, num_layers, batch_first=True, bidirectional=True
        )
        self.dropout = dropout
        self.linear1 = nn.Linear(hidden_size*2, hidden_size)
        self.linear2 = nn.Linear(hidden_size, num_classes)
        self.linear3 = nn.Linear(hidden_size*2, hidden_size)
        self.linear4 = nn.Linear(hidden_size, 9)

    def forward(self, x):
        imageLevelOutputs = []
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).cuda()
        #c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).cuda()

        out, h_n = self.GRU(x, h0)
        
        for i, out_t in enumerate(out.chunk(out.size(1), dim=1)):
            out_t = out_t.squeeze(1)
            out_t = F.relu(self.linear1(out_t))
            out_t = F.dropout(out_t, p=self.dropout)
            out_t = self.linear2(out_t)
            imageLevelOutputs += [out_t]
        imageLevelOutputs = torch.stack(imageLevelOutputs, 1).squeeze(2)
        
        h_n = h_n.view(1,-1)
        studyLevelOutputs = F.relu(self.linear3(h_n))
        studyLevelOutputs = F.dropout(studyLevelOutputs, p=self.dropout)
        studyLevelOutputs = self.linear4(studyLevelOutputs)
        
        return (imageLevelOutputs, studyLevelOutputs)

In [ ]:
seq = BiGRU(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES, dropout=0.2).cuda()

# batch,seqNum,features
sampleInput = torch.rand((1,120,64)).cuda()
sampleImgOutput, sampleStdOutput = seq(sampleInput)
print(sampleImgOutput.size())
print(sampleStdOutput.size())

In [ ]:
def customLoss(imageLevelOutputLogits, imageLevelLabels, studyLeveloutputLogits, studyLevelLabels):
    imageLevelLoss = F.binary_cross_entropy_with_logits(imageLevelOutputLogits,imageLevelLabels,pos_weight = torch.tensor([2.0]).cuda())
    
    studyLevelLoss = 0
    weightList = [0.0736196319, 0.09202453988, 0.1042944785, 0.1042944785, 0.1877300613, 0.06257668712, 0.06257668712, 0.2346625767, 0.0782208589]
    for eachInd in range(9):
        studyLevelLoss += weightList[eachInd]*F.binary_cross_entropy_with_logits(studyLeveloutputLogits[:,eachInd],studyLevelLabels[:,eachInd],reduction='sum')
    
    return imageLevelLoss+studyLevelLoss

In [ ]:
optimizer = optim.Adam(seq.parameters(), lr=1e-4)

In [ ]:
def train_loop(model, train_loader):
    train_total = train_correct = train_cost = 0
    seq.train()
    for x, (y_img, y_std) in tqdm(train_loader):
        x = x.cuda()
        y_img = y_img.cuda()
        y_std = y_std.cuda()
        optimizer.zero_grad()
        (o_img, o_std) = seq(x)
        train_total += y_img.size(1)
        train_correct += ((torch.sigmoid(o_img[0,:])>0.5) == (y_img[0,:]>0.5)).sum().item()
        loss = customLoss(o_img, y_img, o_std, y_std)
        loss.backward()
        optimizer.step()
        train_cost += loss.item()
    return train_cost, train_correct/train_total

def valid_loop(model, valid_loader):
    # Evaluate on validation  data 
    val_total = val_correct = val_cost = 0
    seq.eval()
    with torch.no_grad():
        for x_val, (y_val_img, y_val_std) in tqdm(valid_loader):
            x_val = x_val.cuda()
            y_val_img = y_val_img.cuda()
            y_val_std = y_val_std.cuda()
            (o_val_img, o_val_std) = seq(x_val)
            val_total += y_val_img.size(1)
            val_correct += ((torch.sigmoid(o_val_img[0,:])>0.5) == (y_val_img[0,:]>0.5)).sum().item()
            loss = customLoss(o_val_img, y_val_img, o_val_std, y_val_std)
            val_cost += loss.item()
    return val_cost, val_correct/val_total

def main_loop(n_epochs):
    for epoch in range(n_epochs):
        print('epoch ' + str(epoch) + ':')
        train_avgCost, train_acc = train_loop(seq, trainEmbeddingsDataloader)
        val_avgCost, val_acc = valid_loop(seq, valEmbeddingsDataloader)

        print('train_cost: %.4f, train_acc: %.4f, val_cost: %.4f, val_acc: %.4f'\
              % (train_avgCost, train_acc, val_avgCost, val_acc))
        datestring = datetime.now().strftime("%Y%m%d_%H%M")
        modelPath = 'models/embedderModel/ver03_epoch' + str(epoch) + '_' + datestring +'.pth'
        print('saving: ',modelPath)
        torch.save(seq, modelPath)

In [ ]:
main_loop(5)

In [ ]:
seq = torch.load('models/embedderModel/ver03_epoch0_20201006_2004.pth')

In [ ]:
iterVal = iter(valEmbeddingsDataloader)

In [ ]:
# Sanity Check
seq.eval()
with torch.no_grad():
    x,(y_img, _) = next(iterVal)
    x=x.cuda()
    o_img, _ = seq(x)
    pred = torch.sigmoid(o_img)
    for eachIndex in range(pred.size(1)):
        print((pred[0,eachIndex]).type(torch.float).item(), y_img[0, eachIndex].item())

In [ ]:
with torch.no_grad():
    x,(_, y_std) = next(iterVal)
    x=x.cuda()
    _, o_std = seq(x)
    pred = torch.sigmoid(o_std)
    print(((pred[0]).type(torch.float).cpu().numpy()))
    print(y_std[0].cpu().numpy())